# Import the Libraries

In [ ]:
#Dataframe manipulation
import pandas as pd
import numpy as np

#Data download from Yahoo Finance
from yahooquery import Ticker

#Libraries for the Plotting
import holoviews as hv
from holoviews import opts, dim

hv.extension('bokeh')

#Librarie to save the plots to html object
import panel as pn

# Historical Data for the S&P 500

In [ ]:
#Download the Data from Yahoo Finance
tickers = Ticker('^SPX')
sp500_price = tickers.history(period = 'max')

In [ ]:
#Rearrange the data
sp500_price = sp500_price.reset_index()
sp500_price = sp500_price[['date', 'close']]
sp500_price = sp500_price.set_index('date')
sp500_price.index = pd.to_datetime(sp500_price.index)
sp500_price = pd.DataFrame(sp500_price['close'].resample('M').last())

# Converting the Data into Halves (1st and 2nd Semester)

In [ ]:
#Create the Dataframe for the Division in Halves
sp500_half_df = pd.DataFrame()
sp500_half_df['Year'] = pd.to_datetime(sp500_price.index).year.unique()
sp500_half_df = sp500_half_df[1:]

# Calculate the H1 column
h1_values = []
h2_values = []

for year in sp500_half_df['Year']:
    try:
        start_date_h1 = pd.to_datetime(f"{year-1}-12-31")
        end_date_h1 = pd.to_datetime(f"{year}-06-30")
        start_price_h1 = sp500_price.loc[start_date_h1, 'close']
        end_price_h1 = sp500_price.loc[end_date_h1, 'close']
        h1 = round((end_price_h1 / start_price_h1 - 1) * 100, 2)
        h1_values.append(h1)
    except:
        h1 = np.nan
        h1_values.append(h1)   
        
        
for year in sp500_half_df['Year']:
    try:
        start_date_h2 = pd.to_datetime(f"{year}-06-30")
        end_date_h2 = pd.to_datetime(f"{year}-12-31")
        start_price_h2 = sp500_price.loc[start_date_h2, 'close']
        end_price_h2 = sp500_price.loc[end_date_h2, 'close']
        h2 = round((end_price_h2 / start_price_h2 - 1) * 100, 2)
        h2_values.append(h2)
    except:
        h2 = np.nan
        h2_values.append(h2)      
    
sp500_half_df['H1'] = h1_values
sp500_half_df['H2'] = h2_values

In [ ]:
#Create Pos/Neg Columns for H1 and H2 
sp500_half_df['H1_pos'] = sp500_half_df['H1'].apply(lambda x: x if x > 0 else 0)
sp500_half_df['H1_neg'] = sp500_half_df['H1'].apply(lambda x: x if x < 0 else 0)

sp500_half_df['H2_pos'] = sp500_half_df['H2'].apply(lambda x: x if x > 0 else 0)
sp500_half_df['H2_neg'] = sp500_half_df['H2'].apply(lambda x: x if x < 0 else 0)

# Creating the Graph with Years and Halves (1st and 2nd Semester) Returns

In [ ]:
# Create the horizontal bar chart for H1
last_h1_value = sp500_half_df.iloc[-1]['H1']

vertical_line = hv.HLine(last_h1_value).opts(color='black', line_width=1.5, line_dash='dashed')

#Create the Bar Chart for H1
bar_chart_h1_pos = hv.Bars(sp500_half_df, 'Year', 'H1_pos').opts(
    opts.Bars(invert_axes=True, color='green', width=400, height=1000, tools=['hover'], show_grid=True))

bar_chart_h1_neg = hv.Bars(sp500_half_df, 'Year', 'H1_neg').opts(
    opts.Bars(invert_axes=True, color='red', width=400, height=1000, tools=['hover'], show_grid=True))

bar_chart_h1 = hv.Overlay(bar_chart_h1_pos * bar_chart_h1_neg * vertical_line).opts(
    title = 'S&P 500® Index: 1st Half Annual Returns', ylabel = '1st Half Return (%)')

#Create the Bar Chart for H2
bar_chart_h2_pos = hv.Bars(sp500_half_df, 'Year', 'H2_pos').opts(
    opts.Bars(invert_axes=True, color='green', width=400, height=1000, tools=['hover'], show_grid=True))

bar_chart_h2_neg = hv.Bars(sp500_half_df, 'Year', 'H2_neg').opts(
    opts.Bars(invert_axes=True, color='red', width=400, height=1000, tools=['hover'], show_grid=True))

bar_chart_h2 = hv.Overlay(bar_chart_h2_pos * bar_chart_h2_neg).opts(
    title = 'S&P 500® Index: 2nd Half Annual Returns', ylabel = '2nd Half Return (%)')

#Join Both Charts in a Layout
sp500_full_chart = bar_chart_h1 + bar_chart_h2

In [ ]:
#Save the Plot
p_full = pn.panel(sp500_full_chart)
p_full.save('SP500_full_graph.html', embed = True)

# Calculate the Statistics and Create the Chart with Statistics

In [ ]:
# Calculate average of 'H1' and 'H2'
avg_h1 = round(sp500_half_df['H1'].mean(), 2)
avg_h2 = round(sp500_half_df['H2'].mean(), 2)

# Calculate average of 'H1' and 'H2' when return above 10%
avg_above10_h1 = round(sp500_half_df[sp500_half_df['H1'] > 10]['H1'].mean(), 2)
avg_above10_h2 = round(sp500_half_df[sp500_half_df['H1'] > 10]['H2'].mean(), 2)

# Calculate average of 'H1' and 'H2' when return above 15%
avg_above15_h1 = round(sp500_half_df[sp500_half_df['H1'] > 15]['H1'].mean(), 2)
avg_above15_h2 = round(sp500_half_df[sp500_half_df['H1'] > 15]['H2'].mean(), 2)

# Calculate maximum of 'H1' and corresponding 'H2' value
max_h1 = sp500_half_df['H1'].max()
max_h1_year = sp500_half_df.loc[sp500_half_df['H1'] == max_h1, 'Year'].values[0]
max_h2 = sp500_half_df.loc[sp500_half_df['H1'] == max_h1, 'H2'].values[0]

# Calculate minimum of 'H1' and corresponding 'H2' value
min_h1 = sp500_half_df['H1'].min()
min_h1_year = sp500_half_df.loc[sp500_half_df['H1'] == min_h1, 'Year'].values[0]
min_h2 = sp500_half_df.loc[sp500_half_df['H1'] == min_h1, 'H2'].values[0]

# Create the sp500_half_stats_df DataFrame
data = {
    'Stats.': ['Avg.', 'Avg. (H1 above 10%)', 'Avg. (H1 above 15%)', f'Max. ({max_h1_year})', f'Min. ({min_h1_year})'],
    'H1': [avg_h1, avg_above10_h1, avg_above15_h1, max_h1, min_h1],
    'H2': [avg_h2,  avg_above10_h2, avg_above15_h2, max_h2, min_h2]
}

sp500_half_stats_df = pd.DataFrame(data)

In [ ]:
#Create Pos/Neg Columns for H1 and H2 
sp500_half_stats_df['H1_pos'] = sp500_half_stats_df['H1'].apply(lambda x: x if x > 0 else 0)
sp500_half_stats_df['H1_neg'] = sp500_half_stats_df['H1'].apply(lambda x: x if x < 0 else 0)

sp500_half_stats_df['H2_pos'] = sp500_half_stats_df['H2'].apply(lambda x: x if x > 0 else 0)
sp500_half_stats_df['H2_neg'] = sp500_half_stats_df['H2'].apply(lambda x: x if x < 0 else 0)

In [ ]:
#Change the order for graphical purposes
sp500_half_stats_df = sp500_half_stats_df[::-1]

#Create the Bar Chart for H1 Statistics
bar_chart_h1_stats_pos = hv.Bars(sp500_half_stats_df, 'Stats.', 'H1_pos').opts(
    opts.Bars(invert_axes=True, color='green', width=400, height=250, tools=['hover'], show_grid=True))

bar_chart_h1_stats_neg = hv.Bars(sp500_half_stats_df, 'Stats.', 'H1_neg').opts(
    opts.Bars(invert_axes=True, color='red', width=400, height=250, tools=['hover'], show_grid=True))

bar_chart_h1_stats = hv.Overlay(bar_chart_h1_stats_pos * bar_chart_h1_stats_neg).opts(
    title = 'S&P 500® Index: 1st Half Stats.', ylabel = '1st Half Return (%)')

#Create the Bar Chart for H2 Statistics
bar_chart_h2_stats_pos = hv.Bars(sp500_half_stats_df, 'Stats.', 'H2_pos').opts(
    opts.Bars(invert_axes=True, color='green', width=400, height=250, tools=['hover'], show_grid=True))

bar_chart_h2_stats_neg = hv.Bars(sp500_half_stats_df, 'Stats.', 'H2_neg').opts(
    opts.Bars(invert_axes=True, color='red', width=400, height=250, tools=['hover'], show_grid=True))

bar_chart_h2_stats = hv.Overlay(bar_chart_h2_stats_pos * bar_chart_h2_stats_neg).opts(
    title = 'S&P 500® Index: 2nd Half Stats.', ylabel = '2nd Half Return (%)')

#Join Both Charts in a Layout
sp500_stats_chart = bar_chart_h1_stats + bar_chart_h2_stats

In [ ]:
#Save the Plot
p_stats = pn.panel(sp500_stats_chart)
p_stats.save('SP500_stats_graph.html', embed = True)

# Create a Function so that it is Simpler to Create the Charts

In [ ]:
def index_half_year_return_charts(index_ticker, index_title, index_filename):
    #Download the Data from Yahoo Finance
    tickers = Ticker(index_ticker)
    index_price = tickers.history(period = 'max')

    #Rearrange the data
    index_price= index_price.reset_index()
    index_price= index_price[['date', 'close']]
    index_price= index_price.set_index('date')
    index_price.index = pd.to_datetime(index_price.index)
    index_price= pd.DataFrame(index_price['close'].resample('M').last())

    #Create the Dataframe for the Division in Halves
    index_half_df = pd.DataFrame()
    index_half_df['Year'] = pd.to_datetime(index_price.index).year.unique()
    index_half_df = index_half_df[1:]

    # Calculate the H1 column
    h1_values = []
    h2_values = []

    for year in index_half_df['Year']:
        try:
            start_date_h1 = pd.to_datetime(f"{year-1}-12-31")
            end_date_h1 = pd.to_datetime(f"{year}-06-30")
            start_price_h1 = index_price.loc[start_date_h1, 'close']
            end_price_h1 = index_price.loc[end_date_h1, 'close']
            h1 = round((end_price_h1 / start_price_h1 - 1) * 100, 2)
            h1_values.append(h1)
        except:
            h1 = np.nan
            h1_values.append(h1)   


    for year in index_half_df['Year']:
        try:
            start_date_h2 = pd.to_datetime(f"{year}-06-30")
            end_date_h2 = pd.to_datetime(f"{year}-12-31")
            start_price_h2 = index_price.loc[start_date_h2, 'close']
            end_price_h2 = index_price.loc[end_date_h2, 'close']
            h2 = round((end_price_h2 / start_price_h2 - 1) * 100, 2)
            h2_values.append(h2)
        except:
            h2 = np.nan
            h2_values.append(h2)      

    index_half_df['H1'] = h1_values
    index_half_df['H2'] = h2_values

    #Create Pos/Neg Columns for H1 and H2 
    index_half_df['H1_pos'] = index_half_df['H1'].apply(lambda x: x if x > 0 else 0)
    index_half_df['H1_neg'] = index_half_df['H1'].apply(lambda x: x if x < 0 else 0)

    index_half_df['H2_pos'] = index_half_df['H2'].apply(lambda x: x if x > 0 else 0)
    index_half_df['H2_neg'] = index_half_df['H2'].apply(lambda x: x if x < 0 else 0)

    # Create the horizontal bar chart for H1
    last_h1_value = index_half_df.iloc[-1]['H1']

    vertical_line = hv.HLine(last_h1_value).opts(color='black', line_width=1.5, line_dash='dashed')

    #Create the Bar Chart for H1
    bar_chart_h1_pos = hv.Bars(index_half_df, 'Year', 'H1_pos').opts(
        opts.Bars(invert_axes=True, color='green', width=400, height=1000, tools=['hover'], show_grid=True))

    bar_chart_h1_neg = hv.Bars(index_half_df, 'Year', 'H1_neg').opts(
        opts.Bars(invert_axes=True, color='red', width=400, height=1000, tools=['hover'], show_grid=True))

    bar_chart_h1 = hv.Overlay(bar_chart_h1_pos * bar_chart_h1_neg * vertical_line).opts(
        title = index_title+': 1st Half Annual Returns', ylabel = '1st Half Return (%)')

    #Create the Bar Chart for H2
    bar_chart_h2_pos = hv.Bars(index_half_df, 'Year', 'H2_pos').opts(
        opts.Bars(invert_axes=True, color='green', width=400, height=1000, tools=['hover'], show_grid=True))

    bar_chart_h2_neg = hv.Bars(index_half_df, 'Year', 'H2_neg').opts(
        opts.Bars(invert_axes=True, color='red', width=400, height=1000, tools=['hover'], show_grid=True))

    bar_chart_h2 = hv.Overlay(bar_chart_h2_pos * bar_chart_h2_neg).opts(
        title = index_title+': 2nd Half Annual Returns', ylabel = '2nd Half Return (%)')

    #Join Both Charts in a Layout
    index_full_chart = bar_chart_h1 + bar_chart_h2

    #Save the Plot
    p_full = pn.panel(index_full_chart)
    p_full.save(index_filename+'_full_graph.html', embed = True)

    # Calculate average of 'H1' and 'H2'
    avg_h1 = round(index_half_df['H1'].mean(), 2)
    avg_h2 = round(index_half_df['H2'].mean(), 2)

    # Calculate average of 'H1' and 'H2' when return above 10%
    avg_above10_h1 = round(index_half_df[index_half_df['H1'] > 10]['H1'].mean(), 2)
    avg_above10_h2 = round(index_half_df[index_half_df['H1'] > 10]['H2'].mean(), 2)

    # Calculate average of 'H1' and 'H2' when return above 15%
    avg_above15_h1 = round(index_half_df[index_half_df['H1'] > 15]['H1'].mean(), 2)
    avg_above15_h2 = round(index_half_df[index_half_df['H1'] > 15]['H2'].mean(), 2)

    # Calculate maximum of 'H1' and corresponding 'H2' value
    max_h1 = index_half_df['H1'].max()
    max_h1_year = index_half_df.loc[index_half_df['H1'] == max_h1, 'Year'].values[0]
    max_h2 = index_half_df.loc[index_half_df['H1'] == max_h1, 'H2'].values[0]

    # Calculate minimum of 'H1' and corresponding 'H2' value
    min_h1 = index_half_df['H1'].min()
    min_h1_year = index_half_df.loc[index_half_df['H1'] == min_h1, 'Year'].values[0]
    min_h2 = index_half_df.loc[index_half_df['H1'] == min_h1, 'H2'].values[0]

    # Create the index_half_stats_df DataFrame
    data = {
        'Stats.': ['Avg.', 'Avg. (H1 above 10%)', 'Avg. (H1 above 15%)', f'Max. ({max_h1_year})', f'Min. ({min_h1_year})'],
        'H1': [avg_h1, avg_above10_h1, avg_above15_h1, max_h1, min_h1],
        'H2': [avg_h2,  avg_above10_h2, avg_above15_h2, max_h2, min_h2]
    }

    index_half_stats_df = pd.DataFrame(data)

    #Create Pos/Neg Columns for H1 and H2 
    index_half_stats_df['H1_pos'] = index_half_stats_df['H1'].apply(lambda x: x if x > 0 else 0)
    index_half_stats_df['H1_neg'] = index_half_stats_df['H1'].apply(lambda x: x if x < 0 else 0)

    index_half_stats_df['H2_pos'] = index_half_stats_df['H2'].apply(lambda x: x if x > 0 else 0)
    index_half_stats_df['H2_neg'] = index_half_stats_df['H2'].apply(lambda x: x if x < 0 else 0)

    #Change the order for graphical purposes
    index_half_stats_df = index_half_stats_df[::-1]

    #Create the Bar Chart for H1 Statistics
    bar_chart_h1_stats_pos = hv.Bars(index_half_stats_df, 'Stats.', 'H1_pos').opts(
        opts.Bars(invert_axes=True, color='green', width=400, height=250, tools=['hover'], show_grid=True))

    bar_chart_h1_stats_neg = hv.Bars(index_half_stats_df, 'Stats.', 'H1_neg').opts(
        opts.Bars(invert_axes=True, color='red', width=400, height=250, tools=['hover'], show_grid=True))

    bar_chart_h1_stats = hv.Overlay(bar_chart_h1_stats_pos * bar_chart_h1_stats_neg).opts(
        title = index_title+': 1st Half Stats.', ylabel = '1st Half Return (%)')

    #Create the Bar Chart for H2 Statistics
    bar_chart_h2_stats_pos = hv.Bars(index_half_stats_df, 'Stats.', 'H2_pos').opts(
        opts.Bars(invert_axes=True, color='green', width=400, height=250, tools=['hover'], show_grid=True))

    bar_chart_h2_stats_neg = hv.Bars(index_half_stats_df, 'Stats.', 'H2_neg').opts(
        opts.Bars(invert_axes=True, color='red', width=400, height=250, tools=['hover'], show_grid=True))

    bar_chart_h2_stats = hv.Overlay(bar_chart_h2_stats_pos * bar_chart_h2_stats_neg).opts(
        title = index_title+': 2nd Half Stats.', ylabel = '2nd Half Return (%)')

    #Join Both Charts in a Layout
    index_stats_chart = bar_chart_h1_stats + bar_chart_h2_stats

    #Save the Plot
    p_stats = pn.panel(index_stats_chart)
    p_stats.save(index_filename+'_stats_graph.html', embed = True)

In [ ]:
#Run function for NADAQ Composite
nasdaq_ticker = '^NDX'
nasdaq_title = 'NASDAQ 100'
nasdaq_filename = 'NASDAQ100'

index_half_year_return_charts(index_ticker = nasdaq_ticker, index_title = nasdaq_title, index_filename = nasdaq_filename)

#Run function for Euro Stoxx
eurostoxx_ticker = '^STOXX'
eurostoxx_title = 'Euro Stoxx 600®'
eurostoxx_filename = 'EuroStoxx600'

index_half_year_return_charts(index_ticker = eurostoxx_ticker, index_title = eurostoxx_title, index_filename = eurostoxx_filename)

#Run function for ACWI
acwi_ticker = 'ACWI'
acwi_title = 'MSCI ACWI Index*'
acwi_filename = 'ACWI'

index_half_year_return_charts(index_ticker = acwi_ticker, index_title = acwi_title, index_filename = acwi_filename)